# Notebook for M3 assesment

This is a mock-up notebook for M3 deliverable for AIRGo project. The objective of the notebook is to showcase the end-to-end AIRGo pipeline that will be implemented in the project, illustrated on the sandbox environment provided by Grid2Op. Where relevant, mentions to how the pipeline will be different for AIRgo are provided.

### Import of library

In [ ]:
import grid2op
from grid2op.PlotGrid import PlotMatplot
from grid2op.Backend.PandaPowerBackend import PandaPowerBackend
from grid2op.Agent import DoNothingAgent
from grid2op.Episode import EpisodeData
import numpy as np
import os
import shutil
from grid2op.gym_compat import GymEnv
from gym import Env
from gym.utils.env_checker import check_env
import tqdm
from grid2op.Runner import Runner

### Create a Grid2op environment

Here we load the l2rpn_case14_sandbox file, in the context of our project it should be France network as a whole for the final demonstration.

As you can see for the experience to be reproducible we can set a seed so the train/val/test sets are always the same. 

<strong>The backend will be changed from PandaPowerBackend to PypowsyblBackend</strong> that is the cornerstone of the AIRGo project.

The make function is highly customizable and a lot of parameters could be changed as well other classes.
For more details: https://grid2op.readthedocs.io/en/latest/makeenv.html#grid2op.MakeEnv.make 

In [ ]:
env = grid2op.make("l2rpn_case14_sandbox",backend = PandaPowerBackend()) 
max_iter = 5  # we limit the number of iterations to reduce computation time. Put -1 if you don't want to limit it
env.seed(42)
obs = env.reset()

To create your train, val and test environment. Only needs to be run once !

In [ ]:
try :
    nm_env_train, nm_env_val, nm_env_test = env.train_val_split_random(pct_val=1., pct_test=1., add_for_test="test")
except Exception as e:
    print("Train repositories already exist next to "+ str(env._init_env_path)+"! If you want so, you can delete val, train and test ones.")
    nm_env_train = str(env._init_env_path + "_train")
    nm_env_val = str(env._init_env_path + "_val")
    nm_env_test = str(env._init_env_path + "_test")

    

In [ ]:
train_env = grid2op.make("l2rpn_case14_sandbox_train")

### Then, we can visualize our network and the data associated with each node

In [ ]:
plot_helper = PlotMatplot(train_env.observation_space)
_ = plot_helper.plot_layout()

In [ ]:
_ = plot_helper.plot_obs(obs)

### Different type of actions

<strong>There are five main types of possible actions</strong>:
* Injection actions
* Connection/Deconnection of a line
* Topological configuration at every substation  

     <em>If the correct parameters are given</em>
* Redispatching
* Curtailment



For more details: https://grid2op.readthedocs.io/en/latest/action.html

### Create an agent

An agent corresponds to an algorithm aimed at taking some actions (i.e. the actions mentioned above), regarding some observations on the grid and the possible rewards.

For the purpose of this illustration we chose the DoNothingAgent which takes no action at any time step of the simulation. This agent is already pre-implemented. It is also possible to create one by following the Grid2op framework instructions and rules.

For further information: https://grid2op.readthedocs.io/en/latest/agent.html

In the final demonstration, an RL agent will be trained.

This DoNothingAgent might be replaced by your personnal RL agent.

In [ ]:
my_agent = DoNothingAgent

### Train an agent

We use the training environment to carry out the training phase on the model. 
It is also possible to use a complete gym environment.  

For more details: https://grid2op.readthedocs.io/en/latest/gym.html 

In [ ]:
gym_env = GymEnv(env)

Every possible action can be listed:

In [ ]:
gym_env.action_space

As well as possible observations:

In [ ]:
gym_env.observation_space

These can be modified to fit a more classical form of reinforcment learning algorithms that deal with discrete action space by using:

```python
from grid2op.gym_compat import DiscreteActSpace
gym_env.action_space = DiscreteActSpace(training_env.action_space,
                                        attr_to_keep=["set_bus" , "set_line_status_simple"])
```  
and  
```python
from grid2op.gym_compat import BoxGymObsSpace
gym_env.observation_space = BoxGymObsSpace(training_env.observation_space,
                                           attr_to_keep=["rho"])
gym_env.observation_space
```

Because our agent 'DoNothingAgent' cannot be trained, hereafter is an example of how this could be done with a neural network.

Once you have your trainable agent, you can run some learning iterations by using: 
```python
from YOUR_PACKAGE import YOUR_MODEL
nn_model = YOUR_MODEL(env=gym_env,
               learning_rate=1e-3,
               policy="YOUR_POLICY",
               policy_kwargs={"net_arch": [100, 100, 100]}, # Just an example of architecture
               n_steps=2,
               batch_size=8,
               verbose=True,
               )
```  
and
```python
nn_model.learn(total_timesteps=LEARNING_ITERATION)
```

### Evaluate your agent

In [ ]:
save_path = "saved_agent_DoNothingAgent"
path_save_results = "{}_results".format(save_path)
shutil.rmtree(path_save_results, ignore_errors=True)


runner = Runner(**env.get_params_for_runner(),
                agentClass=my_agent
               )
res = runner.run(nb_episode=1, 
                 max_iter=max_iter,
#                  pbar=tqdm,
                 path_save=f"./{path_save_results}")

In [ ]:
print("The results for DoNothing agent are:")
for _, chron_name, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics with id {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

In [ ]:
os.listdir(path_save_results)
EpisodeData.list_episode(path_save_results)


In [ ]:
all_episodes = EpisodeData.list_episode(path_save_results)
this_episode = EpisodeData.from_disk(*all_episodes[0])
li_actions = this_episode.actions

Extraction of all the actions taken by the agent

In [ ]:
for act in li_actions:
    dict_act_ = act.as_dict()

In [ ]:
dict_act_

As you can see, the dictionnary containing these actions is empty, which is rather logical since the agent does not take any action.

We can now check certain observation values for the episode. Typically the state of the lines (connected/disconnected) at each stage or the number of actual disconnections.

In [ ]:
li_observations = this_episode.observations
nb_real_disc = 0
for obs_ in li_observations:
    nb_real_disc += (obs_.line_status == False).sum()
print(f'Total number of disconnected powerlines cumulated over all the timesteps : {nb_real_disc}')

In [ ]:
actions_count = {}
for act in li_actions:
    act_as_vect = tuple(act.to_vect())
    if not act_as_vect in actions_count:
        actions_count[act_as_vect] = 0
    actions_count[act_as_vect] += 1
print("The agent did {} different valid actions:\n".format(len(actions_count)))

In [ ]:
for act in li_actions:
    print(act)